Step 1: Dataset Preparation
Before training, ensure your dataset is ready. Typically, you would:

Serialize your multilayer structures into a format your model can process.
Split your dataset into training and validation sets.
Normalize or standardize your spectral data if necessary.
Use a DataLoader to batch and shuffle your data.

In [ ]:
from spectra_layer.model.transformer import SpectraTransformer
from spectra_layer.model.utils import load_pretrained_weights, save_model, load_model
# Other imports as necessary
from torch.utils.data import DataLoader, TensorDataset
import torch

# Example dataset preparation
X_train, y_train = ... # Your training data and labels
X_val, y_val = ... # Your validation data and labels

train_dataset = TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float())
val_dataset = TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).float())

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


Step 2: Model Initialization
Instantiate your model, loss function, and optimizer. Assuming a regression task (predicting continuous spectral values), you might use the Mean Squared Error (MSE) loss and an optimizer like Adam.

In [ ]:
from spectra_layer.model.transformer import SpectraTransformer
import torch.optim as optim
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SpectraTransformer(ntoken=..., ninp=..., nhead=..., nhid=..., nlayers=...).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


Step 3: Training Loop
Define the training loop where your model learns from the data. This typically involves iterating over the dataset multiple times (epochs), processing each batch, calculating the loss, and updating the model parameters.

In [ ]:
def train(epoch):
    model.train()
    total_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if batch_idx % LOG_INTERVAL == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
    print(f'====> Epoch: {epoch} Average loss: {total_loss / len(train_loader.dataset):.4f}')


Step 4: Validation
After each epoch, evaluate your model on the validation set to monitor its performance on unseen data.

In [ ]:
def validate():
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += criterion(output, target).item()  # sum up batch loss

    val_loss /= len(val_loader.dataset)
    print(f'\nValidation set: Average loss: {val_loss:.4f}\n')


Step 5: Training Execution
Execute the training and validation process.
Step 6: Making Predictions
Once your model is trained, you can use it to make predictions. Ensure the input data is prepared in the same way as your training data.

In [ ]:
for epoch in range(1, EPOCHS + 1):
    train(epoch)
    validate()
def predict(input_data):
    model.eval()
    with torch.no_grad():
        input_data = input_data.to(device)
        prediction = model(input_data)
        return prediction

save the model

In [ ]:
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    # Include any other metadata here
    'epochs': epochs,
    'loss': loss,
}, model_path)
